In [81]:
import os
import re

import matplotlib.pyplot as plt
import numpy as np 
import pyrotd
from obspy.core.utcdatetime import UTCDateTime
from obspy.clients.fdsn import Client
import obspy as obs
import pandas as pd
from glob import glob

In [82]:

# Reading the stations catalog file and querying the name and location of the stations
st_file = pd.read_csv('/Users/rshimony/Desktop/WillametteValley/scottsmills_eq/metadata/snr_stations_fixed.csv')
st_names = np.array(st_file['st_nm'])
st_lons = np.array(st_file['st_lon'])
st_lats = np.array(st_file['st_lat'])  

# Taking the unique values (since the stations file contains the 3 components for each station)
st_names_unq = pd.unique(st_names)
st_lons_unq = pd.unique(st_lons)
st_lats_unq = pd.unique(st_lats)

# Setting observed, Stephenson and models (synts) data directories
obs_dir = '/Users/rshimony/Desktop/WillametteValley/scottsmills_eq/waveforms/observed_data_1hz_snr/'
steph_dir = '/Users/rshimony/Desktop/WillametteValley/Models/stephenson_rfile/rfile_steph_scottsmills/'
synts_dir = '/Users/rshimony/Desktop/WillametteValley/Models/final_sims/scottsmills/eugspe_1d/rfile_steph_basin_scottsmills_eugspe_1d/'
# Setting output dir and output flatfile name
outdir = '/Users/rshimony/Desktop/WillametteValley/Models/final_sims/scottsmills/'
ff_name = 'psa_ff_obs_scottsmills.csv'

# Setting traces files
obs_trs = sorted(glob(obs_dir + '*_unfilt.sac'))
synts_trs = sorted(glob(synts_dir + '*.*v'))
steph_trs = sorted(glob(steph_dir + '*.*v'))

# Making arrays of unique station names *from the waveform traces*
obs_nms = []
for i in range(len(obs_trs)):
    obs_nm = obs_trs[i].split('.')[1]
    obs_nms.append(obs_nm)
obs_nms_unq = pd.unique(obs_nms)
    
synt_nms = []
for i in range(len(synts_trs)):   
    synt_nm = (synts_trs[i].split('/')[-1]).split('.')[0]
    synt_nms.append(synt_nm)
synt_nms_unq = pd.unique(synt_nms)

steph_nms = []
for i in range(len(steph_trs)):   
    steph_nm = (steph_trs[i].split('/')[-1]).split('.')[0]
    steph_nms.append(steph_nm)
steph_nms_unq = pd.unique(steph_nms)

# Making lists of strings with the 3 components for each station
obs_strs = []
obs_strs_filt = []
synts_strs = []
steph_strs = []

for i in range(len(st_names_unq)):
    # Check if the station trace is in the stations catalog. Since the location info is not included in 
    # trace stats, the string list have to be alligned with stations catalog order
    if st_names_unq[i] in obs_nms_unq: 
        # Read and append traces
        obs_str = obs.read(obs_dir + '*' + st_names_unq[i] + '*_unfilt.sac')
        obs_str_cp = obs_str.copy()
        obs_strs.append(obs_str_cp)
        # Filter the trace and append
        obs_str_filt = obs_str.copy()
        df_obs = obs_str_filt[0].stats.sampling_rate
        obs_str_filt.filter('lowpass_cheby_2',freq=1.0)
        obs_strs_filt.append(obs_str_filt)
    
    if st_names_unq[i] in synt_nms_unq:
        synts_str = obs.read(synts_dir + st_names_unq[i] + '.*v')
        synts_str_cp = synts_str.copy()
        df_synts = synts_str_cp[0].stats.sampling_rate
        synts_str_filt = synts_str_cp.filter('lowpass_cheby_2',freq=1.0)
        synts_strs.append(synts_str_filt)
        
    if st_names_unq[i] in steph_nms_unq:
        steph_str = obs.read(steph_dir + st_names_unq[i] + '.*v')
        steph_str_cp = steph_str.copy()
        df_steph = steph_str_cp[0].stats.sampling_rate
        steph_str_filt = steph_str_cp.filter('lowpass_cheby_2',freq=1.0)
        steph_strs.append(steph_str_filt)

# Checking if string contains more than 3 components and if so, taking the only the 3 we need
obs_strs_3 = []
obs_strs_filt_3 = []
synts_strs_3 = []
steph_strs_3 = []

for i in obs_strs:
    if len(i) >= 3:
       obs_strs_3.append(i[-3:]) 
       
for i in obs_strs_filt:
    if len(i) >= 3:
       obs_strs_filt_3.append(i[-3:]) 
        
for i in synts_strs:
    if len(i) >= 3:
       synts_strs_3.append(i[-3:]) 
       
for i in steph_strs:
    if len(i) >= 3:
       steph_strs_3.append(i[-3:]) 

In [83]:
# Setting damping% and freqs
osc_damping = 0.05
osc_freqs = [0.1,0.2,0.5,1]

# Calculating PSA for each component in each station and placing the results in a flatfile
counter_psa = 1
for i in range(len(obs_strs_filt_3)):
    if counter_psa % 1 == 0:
        print(str(counter_psa) + ' from ' + str(len(obs_strs_filt_3)))
    for j in range(3):
        # Looping over traces. Taking start time, dt and channel info
        trobs = obs_strs_filt_3[i][j] # This parameter needs to be manually changed for different datasets (observed, steph, synts)
        t_start = trobs.stats.starttime
        dt = trobs.stats.delta
        chan = trobs.stats.channel
        
        # Calculating PSA
        osc_resps = pyrotd.calc_spec_accels(
        dt, trobs.data, osc_freqs, osc_damping)
        
        data_psa = [[st_names_unq[i], chan ,st_lons_unq[i], st_lats_unq[i] , 
                 osc_resps[0][1] , osc_resps[1][1] , osc_resps[2][1] , osc_resps[3][1]], ]
        
        #Setting dataframe and save flatfile
        if i == 0 and j == 0:
            df_psa = pd.DataFrame(data=data_psa,columns=['st_name', 'channel', 'st_lon', 'st_lat'
                                                                   , 'T=10', 'T=5', 'T=2', 'T=1'])
        else:
            df_psa_temp = pd.DataFrame(data=data_psa,columns=['st_name', 'channel', 'st_lon', 'st_lat'
                                                                   , 'T=10', 'T=5', 'T=2', 'T=1'])
            df_psa = pd.concat([df_psa, df_psa_temp], ignore_index=True)
            
    counter_psa = counter_psa + 1
        
df_psa.to_csv(outdir+ff_name,index=False)

1 from 88
2 from 88
3 from 88
4 from 88
5 from 88
6 from 88
7 from 88
8 from 88
9 from 88
10 from 88
11 from 88
12 from 88
13 from 88
14 from 88
15 from 88
16 from 88
17 from 88
18 from 88
19 from 88
20 from 88
21 from 88
22 from 88
23 from 88
24 from 88
25 from 88
26 from 88
27 from 88
28 from 88
29 from 88
30 from 88
31 from 88
32 from 88
33 from 88
34 from 88
35 from 88
36 from 88
37 from 88
38 from 88
39 from 88
40 from 88
41 from 88
42 from 88
43 from 88
44 from 88
45 from 88
46 from 88
47 from 88
48 from 88
49 from 88
50 from 88
51 from 88
52 from 88
53 from 88
54 from 88
55 from 88
56 from 88
57 from 88
58 from 88
59 from 88
60 from 88
61 from 88
62 from 88
63 from 88
64 from 88
65 from 88
66 from 88
67 from 88
68 from 88
69 from 88
70 from 88
71 from 88
72 from 88
73 from 88
74 from 88
75 from 88
76 from 88
77 from 88
78 from 88
79 from 88
80 from 88
81 from 88
82 from 88
83 from 88
84 from 88
85 from 88
86 from 88
87 from 88
88 from 88
